In [3]:
pip install numpy

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [7]:
pip install bs4

     |████████████████████████████████| 128 kB 28.5 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1271 sha256=e3a80e346ded8593a79b0d5365eab31e3a38e8443b83e73c4c781f5d2978b917
  Stored in directory: /root/.cache/pip/wheels/19/f5/6d/a97dd4f22376d4472d5f4c76c7646876052ff3166b3cf71050
Successfully built bs4
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
#Web Access and parsing
import urllib.request, urllib.parse
from bs4 import BeautifulSoup

# Importing Data
netflix_data = pd.read_csv("netflix_titles.csv")
netflix_data.head() #Displaying first few entries
#np.where(pd.isnull(netflix_data.release_year)) #checking for empty values for release years, None found, **might implement later just in case**

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [9]:
#IMDB Access/Search and Parse HTML to grab title_ID
def grab_ID(title,year,media_type):
    encoding = 'utf-8' #standard internet encoding
    find_url = 'https://www.imdb.com/find' #base search URL for IMDB
    values = {'q':title} # What we are searching for
    query = urllib.parse.urlencode(values).encode(encoding) #parsing the query and encoding correctly for web to understand it
    req = urllib.request.Request(find_url,query) #request: the "link" you can paste in the search bar
    resp = urllib.request.urlopen(req) #sending the request and recording the response
    html = BeautifulSoup(resp,"html") #using Beautiful Soup 4 to process the HTML response
    #Finding top 3 results through <td> tags with result text class in the HTML response. <td> is a table cell in HTML
    search = html.findAll("td", {"class": "result_text"})[:3]
    if len(search) > 0:
        for item in search: 
            if  str(item.text).find(str(year)) > 0:
                if str(media_type) == "TV Show":
                    if str(item.text).find("Series") > 0:
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        #confidence = 
                        break
                    else:
                        #confidence = 
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        break
                else:
                    title_names = str(item.text)
                    title_IDs = str(item.a["href"])
                    #confidence =
                    break

            else:
                if str(media_type) == "TV Show":
                    if str(item.text).find("Series") > 0:
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        #confidence =
                        break
                    else:
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        #confidence =
                        break
                else:
                    title_names = "Not Found"
                    title_IDs = "Not Found"
                    #confidence =
                    break
    else:
        title_names = "Not Found"
        title_IDs = "Not Found"
        #confidence =

    return title_names, title_IDs

In [15]:
#Acessing IMDB and parse HTML to grab Ratings
def grab_Rating(title_ID):
    imdb_url = "https://www.imdb.com" #base imdb website
    if title_ID == "Not Found":
        rating = "Not Found"
    else:
        target_url = imdb_url + title_ID #url for title
        target_url = target_url
        resp = urllib.request.urlopen(target_url)
        html = BeautifulSoup(resp,"html") #using Beautiful Soup 4 to process the HTML responce
        search = html.findAll("div", {"class": "ratingValue"})
        if len(search) > 0:
            for item in search: #Grabing rating text
                rating = str(item.strong['title'])
        else:
            rating = "Not Found"

    return rating

In [16]:
IMDB_rating = []
IMDB_titleID = []
IMDB_title_name = []
NumShows = len(netflix_data.index)
for i in range(NumShows):
    title_name, title_ID = grab_ID(netflix_data.title[i],netflix_data.release_year[i],netflix_data.type[i])
    IMDB_titleID.append(title_ID)
    IMDB_title_name.append(title_name)
    rating = grab_Rating(title_ID)
    IMDB_rating.append(rating)
    if i%100 == 0:
        print("Progress is: ", i)
    #print(netflix_data.title[i], "\t\t" title_name,"\t\t" , i)

Progress is:  0
Progress is:  100
Progress is:  200
Progress is:  300
Progress is:  400
Progress is:  500
Progress is:  600
Progress is:  700
Progress is:  800
Progress is:  900
Progress is:  1000
Progress is:  1100
Progress is:  1200
Progress is:  1300
Progress is:  1400
Progress is:  1500
Progress is:  1600
Progress is:  1700
Progress is:  1800
Progress is:  1900
Progress is:  2000
Progress is:  2100
Progress is:  2200
Progress is:  2300
Progress is:  2400
Progress is:  2500
Progress is:  2600
Progress is:  2700
Progress is:  2800
Progress is:  2900
Progress is:  3000
Progress is:  3100
Progress is:  3200
Progress is:  3300
Progress is:  3400
Progress is:  3500
Progress is:  3600
Progress is:  3700
Progress is:  3800
Progress is:  3900
Progress is:  4000
Progress is:  4100
Progress is:  4200
Progress is:  4300
Progress is:  4400
Progress is:  4500
Progress is:  4600
Progress is:  4700
Progress is:  4800
Progress is:  4900
Progress is:  5000


HTTPError: HTTP Error 503: Service Temporarily Unavailable

In [18]:
#Writing outputs
output = pd.DataFrame({'IMDB_titleID': IMDB_titleID, 'IMDB_rating': IMDB_rating, 'IMDB_title_name': IMDB_title_name})
output_name = 'netflix_titles_IMDB_results_' + '2022-05-20' + '.csv'
output.head()
output.to_csv(output_name, index=False)
print("Complete!")

ValueError: arrays must all be same length